<a href="https://colab.research.google.com/github/srijansingh53/hatErase/blob/master/dataset_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import os
os.chdir('/content/drive/My Drive/ML Project/ML-PR')
!ls

dataset  Dataset_Exploration.ipynb  dataset_preprocess.ipynb  outputs


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import re 
import string
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling as pp
import nltk
from nltk.stem import WordNetLemmatizer

In [16]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [31]:
df = pd.read_csv('outputs/final_dataset.csv', encoding='latin-1')
print(df.count())
df.head()

text     196388
label    196388
dtype: int64


,text,label
0,"""like this if you are a tribe fan""",0
1,"""you're idiot.......................""",1
2,"""I am a woman Babs, and the only ""war on women...",1
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1
4,"""haha green me red you now loser whos winning ...",1


In [18]:
report = pp.ProfileReport(df)
report.to_file('outputs/profile_report.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [19]:
df.groupby('label').describe()

text                                                                
        count  unique                                                top freq
label                                                                        
0      155156  155124  By wearing the #BalidaanBadge over his gloves ...    5
1       41232   41212                                  "you're an idiot"    4

### Processing of text

In [0]:
df['hashtags'] = df['text'].apply(lambda x:re.findall('#\w*',x))
df['users'] = df['text'].apply(lambda x:re.findall('@\w*',x))
df['links'] = df['text'].apply(lambda x:re.findall('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+',x, flags=re.MULTILINE))
df['links'] = df['links'].apply(lambda x:list(set(x)))

> For classification based on word embeddings as features, stopwords are not required

In [0]:
def preprocess(text):
    # Removing links- http
    text =  re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', text, flags=re.MULTILINE)
    # Removing mentions and hashtags
    text = re.sub(r"#(\w+)", ' ', text, flags=re.MULTILINE)
    text = re.sub(r"@(\w+)", ' ', text, flags=re.MULTILINE)
    # Removing punctuations
    text = re.sub(r'[^\w\d\s]', ' ', text)
    # convert to lower case
    text = re.sub(r'^\s+|\s+?$', ' ', text.lower())
    # Removing digits
    text = re.sub(r'\d', ' ', text)
    # Removing other symbols
    text = re.sub('[ãâªð³ÂÃ±¤¡¥¶¦§_®¯¹¾²µ½¼º]+', ' ', text)
    # collapse all white spaces
    text = re.sub(r'\s+', ' ', text)
    # remove stop words and perform stemming
    stop_words = nltk.corpus.stopwords.words('english')
    # 
    lemmatizer = WordNetLemmatizer() 
    return ' '.join(
        lemmatizer.lemmatize(term) 
        for term in text.split()
        if term not in set(stop_words)
    )

In [0]:
df['processed_text'] = df.text.apply(lambda row : preprocess(row))

pd.DataFrame(df, columns=['text','label','hashtags', 'users', 'links', 'processed_text']).to_csv('outputs/processed_wos.csv', index = False)

In [37]:
report2 = pp.ProfileReport(df)
report2.to_file('outputs/processed_wos_report.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


> For classification using lstms we need not remove stopwords as model would be based on seq to seq model. 

In [0]:
def preprocess_ws(text):
    # Removing links- http
    text =  re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', text, flags=re.MULTILINE)
    # Removing mentions and hashtags
    text = re.sub(r"#(\w+)", ' ', text, flags=re.MULTILINE)
    text = re.sub(r"@(\w+)", ' ', text, flags=re.MULTILINE)
    # Removing punctuations
    text = re.sub(r'[^\w\d\s]', ' ', text)
    # convert to lower case
    text = re.sub(r'^\s+|\s+?$', ' ', text.lower())
    # Removing digits
    text = re.sub(r'\d', ' ', text)
    # Removing other symbols
    text = re.sub('[ãâªð³ÂÃ±¤¡¥¶¦§_®¯¹¾²µ½¼º]+', ' ', text)
    # collapse all white spaces
    text = re.sub(r'\s+', ' ', text)

    # lemmatizer = WordNetLemmatizer() 
    return ' '.join(term
        # lemmatizer.lemmatize(term) 
        for term in text.split()
        # if term not in set(stop_words)
    )

In [0]:
df['processed_text'] = df.text.apply(lambda row : preprocess_ws(row))

pd.DataFrame(df, columns=['text','label','hashtags', 'users', 'links', 'processed_text']).to_csv('outputs/processed_ws.csv', index = False)

In [45]:
report2 = pp.ProfileReport(df)
report2.to_file('outputs/processed_ws_report.html')

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)
